In [96]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from kafka import KafkaConsumer
import os
import sys
import string
from pyspark.sql import SQLContext
from pyspark import SparkContext
import pandas as pd
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.tuning import CrossValidatorModel
import numpy as np
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [97]:
def createSparkSession():
    spark = SparkSession.builder \
    .master(f"local[*]") \
    .appName("myApp") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') \
    .getOrCreate()

    spark.sparkContext.setLogLevel("WARN")
    return spark


In [98]:
#load the model stored in local
cvModel=CrossValidatorModel.load('trained_model/modelRF')

In [99]:
#kafka connexion setting
kafka_broker_hostname='pidgeot.fib.upc.edu:9092'
kafka_consumer_portno='9092'
kafka_broker=kafka_broker_hostname + ':' + kafka_consumer_portno
kafka_topic_input='bdm_p2'


In [100]:
# Pull data from Kafka topic
consumer = KafkaConsumer(kafka_topic_input,bootstrap_servers=[kafka_broker])

In [102]:
spark=createSparkSession()
maxMessages=10
neighborhood_ids=[]
prices=[]
while True:
    for message in consumer:
        message=message.value.decode("utf-8").split(",")
        neighborhood_ids.append(int(message[1][1:]))
        prices.append(float(message[2]))
        i=+1
        print("Input data neighborhood_id: "+message[1][1:]+" and "+"price: "+message[2])
        if(len(neighborhood_ids)==maxMessages):
            pdf = pd.DataFrame({"neighborhood_id": neighborhood_ids,
                                "price": prices})
            i=0
            new_test = spark.createDataFrame(pdf)
            print("\n")
            print("The predicted ratio of bedrooms to bathrooms for the above 10 inputs:")
            print(cvModel.transform(new_test).select('neighborhood_id','price','Prediction').show())
            print("----------------------------------------------\n")
            neighborhood_ids=[]
            prices=[]



Input data neighborhood_id: 1904302 and price: 798051
Input data neighborhood_id: 2476184 and price: 1050013
Input data neighborhood_id: 3750929 and price: 280073
Input data neighborhood_id: 980253 and price: 319073
Input data neighborhood_id: 3750859 and price: 279039
Input data neighborhood_id: 2476184 and price: 325058
Input data neighborhood_id: 3596096 and price: 460082
Input data neighborhood_id: 3320699 and price: 200021
Input data neighborhood_id: 3750859 and price: 445084
Input data neighborhood_id: 1904302 and price: 659941


The predicted ratio of bedrooms to bathrooms for the above 10 inputs:
+---------------+---------+------------------+
|neighborhood_id|    price|        Prediction|
+---------------+---------+------------------+
|        1904302| 798051.0|0.8347211993050809|
|        2476184|1050013.0|0.7413550895161769|
|        3750929| 280073.0|0.7630455993202705|
|         980253| 319073.0|0.6829056183378455|
|        3750859| 279039.0|0.7630455993202705|
|        247

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\Users\xange\anaconda3\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Users\xange\anaconda3\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\xange\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 